<a href="https://colab.research.google.com/github/aswa09/EVA-4/blob/master/S7/EVA4_S7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import sys

## Get Train and Test data



In [0]:
def get_train_test():

  classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog',
    'frog', 'horse', 'ship', 'truck')

  transform = transforms.Compose(
      [transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

  SEED = 1

  # CUDA?
  cuda = torch.cuda.is_available()
  print("CUDA Available?", cuda)

  # For reproducibility
  torch.manual_seed(SEED)

  if cuda:
    torch.cuda.manual_seed(SEED)

  # dataloader arguments - something you'll fetch these from cmdprmt
  dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

  trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform)

  testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform)

  trainloader = torch.utils.data.DataLoader(trainset, **dataloader_args)
  testloader = torch.utils.data.DataLoader(testset, **dataloader_args)

  return trainloader, testloader

##Base Model

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.dropout_value = 0.1

        # Input Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        ) # In: 32x32x3 | Out: 32x32x32 | RF: 3

        # CONVOLUTION BLOCK 1
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU()
        ) # In: 32x32x32 | Out: 32x32x32 | RF: 5

        # TRANSITION BLOCK 1
        self.pool1 = nn.MaxPool2d(2, 2) # In: 32x32x32 | Out: 16x16x12 | RF: 6

        # Depthwise Separable Convolution
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), padding=1, groups=32, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        ) # In: 16x16x32 | Out: 16x16x32 | RF: 10

        # CONVOLUTION BLOCK 2
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        ) # In: 16x16x32 | Out: 16x16x64 | RF: 14

        # TRANSITION BLOCK 1
        self.pool2 = nn.MaxPool2d(2, 2) # In: 16x16x32 | Out: 8x8x32 | RF: 16

        # Dilated Convolution
        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, dilation=2, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        ) # In: 8x8x64 | Out: 8x8x64 | RF: 24
        
        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(self.dropout_value)
        ) # In: 8x8x64 | Out: 8x8x128 | RF: 32

        # OUTPUT BLOCK
        self.gap = nn.Sequential(
            nn.AdaptiveAvgPool2d(output_size=1)
        ) # In: 8x8x128 | Out: 1x1x128 | RF: 40
        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=10, kernel_size=(1, 1), padding=0, bias=False)
            # nn.ReLU() NEVER!
        ) # In: 1x1x128 | Out: 1x1x10 | RF: 56


    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.pool1(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.pool2(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.gap(x)
        x = self.convblock7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

## Training

In [0]:
def train(model, device, train_loader, optimizer, epoch):
  train_losses = []
  train_acc = []

  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()

    # Predict
    y_pred = model(data)

    L1 = 0
    #loss
    loss = F.nll_loss(y_pred, target)
    if L1 > 0:
      l1_crit = nn.L1Loss(size_average = False).to(device)
      l1_loss = 0
      for param in model.parameters():
        zero_vector = torch.rand_like(param) * 0
        l1_loss += l1_crit(param, zero_vector)
      factor = 0.0005
      loss += factor * l1_loss
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)
    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)
  return train_losses,train_acc

##Testing

In [0]:
def test(model, device, test_loader):
  try:
    model.eval()
    test_loss = 0
    correct = 0
    test_losses=[]
    test_acc = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_losses.append(test_loss)
    test_acc.append(100. * correct / len(test_loader.dataset))
  except Exception as e:
    print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno) + " " + type(e).__name__ + " " + str(e))
    sys.exit(1)
  return test_losses,test_acc

##Plotting Test Accuracies and Test Losses

In [0]:
def plot_acc_loss(test_losses, test_acc):
  try:
  
    fig, (ax1, ax2) = plt.subplots(2, figsize=(15,10))
    fig.suptitle('Test Loss and Test Accuracy for the models', fontsize=16)
    ax1.plot(test_losses[0])

    ax1.set_title("Test Loss")
    
    ax2.plot(test_acc[0])

    ax2.set_title("Test Accuracy")
   
    plt.savefig('acc_vs_loss.jpg')
    plt.show()
  except Exception as e:
        print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno) + " " + type(e).__name__ + " " + str(e))
        sys.exit(1)

##Main function


In [0]:
def main():
  !pip install torchsummary
  from torchsummary import summary
  from tqdm import tqdm
  
  train_loader, test_loader = get_train_test()

  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  print(device)

  model = Net().to(device)
  summary(model, input_size=(3, 32, 32))
  optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
  EPOCHS=2
 
  test_loss=[]
  test_accuracy=[]
  for epoch in range(EPOCHS):
      print("EPOCH:", epoch + 1)
      train_losses,train_acc=train(model, device, train_loader, optimizer, epoch=EPOCHS)
      test_losses,test_acc=test(model, device, test_loader)

      test_loss.append(test_losses)
      test_accuracy.append(test_acc)
  
  plot_acc_loss(test_losses,test_acc)

##Calling Main

In [0]:
if __name__ == '__main__':
  from tqdm import tqdm
  if hasattr(tqdm, '_instances'):
    tqdm._instances.clear()
  main()

CUDA Available? True
Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/391 [00:00<?, ?it/s]

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
           Dropout-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]           9,216
       BatchNorm2d-6           [-1, 32, 32, 32]              64
              ReLU-7           [-1, 32, 32, 32]               0
         MaxPool2d-8           [-1, 32, 16, 16]               0
            Conv2d-9           [-1, 32, 16, 16]             288
      BatchNorm2d-10           [-1, 32, 16, 16]              64
             ReLU-11           [-1, 32, 16, 16]               0
          Dropout-12           [-1, 32, 16, 16]               0
           Conv2d-13           [-1, 64, 16, 16]          18,432
      BatchNorm2d-14           [-1

Loss=1.1840957403182983 Batch_id=390 Accuracy=49.77: 100%|██████████| 391/391 [00:11<00:00, 32.89it/s]
  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 1.1800, Accuracy: 5658/10000 (56.58%)

EPOCH: 2


Loss=0.8185805082321167 Batch_id=390 Accuracy=65.83: 100%|██████████| 391/391 [00:11<00:00, 33.89it/s]



Test set: Average loss: 0.9210, Accuracy: 6771/10000 (67.71%)

Error on line 4 NameError name 'plt' is not defined


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
